# Ausführen eines Trainingsskripts als Befehlsauftrag

Sie können das Python SDK für Azure Machine Learning verwenden, um Skripts als Befehlsaufträge zu übermitteln. Mithilfe von Aufträgen können Sie die Eingabeparameter und Ausgaben beim Trainieren eines Machine Learning-Modells problemlos nachverfolgen.

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Einleiten eines Befehlsauftrags

Führen Sie die folgende Zelle aus, um ein Klassifizierungsmodell zur Vorhersage von Diabetes zu trainieren. Das Modell wird trainiert, indem das Skript **train-model-parameters.py** ausgeführt wird, das sich im Ordner **src** befindet. Es nutzt die Datei **diabetes.csv** als Trainingsdaten. 

- `code`: gibt den Ordner an, der das auszuführende Skript enthält.
- `command`: gibt an, was genau ausgeführt werden soll.
- `environment`: gibt die erforderlichen Pakete an, die in der Compute-Instanz installiert werden müssen, ehe der Befehl ausgeführt wird.
- `compute`: gibt die Compute-Instanz an, die zum Ausführen des Befehls verwendet werden soll.
- `display_name`: der Name des einzelnen Auftrags.
- `experiment_name`: der Name des Experiments, zu dem der Auftrag gehört.

Beachten Sie, dass der Befehl, mit dem Sie das Skript im Terminal testen, identisch mit dem Befehl in der nachfolgenden Konfiguration des Auftrags ist. 

In [ ]:
# enter details of your AML workspace
subscription_id = "YOUR-SUBSCRIPTION-ID"
resource_group = "rg-dp100-labs"
workspace = "mlw-dp100-labs"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)